# 🚀 Advanced Hyperparameter Tuning for Top 3 Performing Models (Speed Optimized)

This notebook performs comprehensive hyperparameter optimization and class imbalance handling for the top 3 models identified from baseline analysis. Each model will be optimized separately with advanced techniques optimized for fast execution.

## Top 3 Models to Optimize:
1. **Gradient Boosting** (AUC-ROC: 0.8390)
2. **CatBoost** (AUC-ROC: 0.8356) 
3. **AdaBoost** (AUC-ROC: 0.8345)

## Advanced Techniques Applied:
- **Class Imbalance**: SMOTE-Tomek (focused hybrid sampling)
- **Optimization**: Optuna Bayesian Search (20 trials per model for speed)
- **Validation**: 5-fold Stratified Cross-Validation
- **Ensemble**: Voting & Stacking Classifiers
- **Goal**: Maximize performance with ultra-fast execution

## 📂 Load Data
Load the feature-engineered training dataset and prepare features (X) and target (y).

In [7]:
# Import comprehensive libraries for advanced techniques
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Core ML libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# Class Imbalance Handling
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from collections import Counter

# Advanced Models
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# Hyperparameter Optimization
import optuna
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import time
import joblib

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
print("📂 Loading feature-engineered training dataset...")
data = pd.read_csv('../Data/output/feature_engineered_train.csv')
print(f'Dataset shape: {data.shape}')

# Separate features and target
X = data.drop(columns=['customerID', 'Churn'])
y = data['Churn']

# Encode target if needed
if y.dtype == 'object' or y.dtype.name == 'category':
    le = LabelEncoder()
    y = le.fit_transform(y)
    print("Target variable encoded (No=0, Yes=1)")

# Analyze class distribution
class_counts = Counter(y)
print(f"\n📊 Class Distribution:")
print(f"Class 0 (No Churn): {class_counts[0]} ({class_counts[0]/len(y)*100:.2f}%)")
print(f"Class 1 (Churn): {class_counts[1]} ({class_counts[1]/len(y)*100:.2f}%)")
print(f"Imbalance Ratio: {class_counts[0]/class_counts[1]:.2f}:1")

# Check for missing values
missing = data.isnull().sum().sum()
print(f'\nMissing values: {missing}')
assert missing == 0, 'There are missing values in the data!'

print(f'\n✅ Data prepared successfully!')
print(f'Features shape: {X.shape}, Target shape: {y.shape}')

# Create train-validation split for proper evaluation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, 
                                                  stratify=y, random_state=42)
print(f'Training set: {X_train.shape}, Validation set: {X_val.shape}')

📂 Loading feature-engineered training dataset...
Dataset shape: (5625, 22)
Target variable encoded (No=0, Yes=1)

📊 Class Distribution:
Class 0 (No Churn): 4130 (73.42%)
Class 1 (Churn): 1495 (26.58%)
Imbalance Ratio: 2.76:1

Missing values: 0

✅ Data prepared successfully!
Features shape: (5625, 20), Target shape: (5625,)
Training set: (4500, 20), Validation set: (1125, 20)


## 🎯 Top 3 Models for Advanced Optimization (Speed Optimized)

Based on baseline model evaluation, we will focus on the top 3 performing models for fast execution:

1. **Gradient Boosting Classifier** (AUC-ROC: 0.8390) - Best overall performer
2. **CatBoost Classifier** (AUC-ROC: 0.8356) - Strong gradient boosting variant  
3. **AdaBoost Classifier** (AUC-ROC: 0.8345) - Adaptive boosting approach

Each model will be optimized separately with SMOTE-Tomek class imbalance technique for ultra-fast execution.

In [8]:
# Install required packages for advanced techniques
import subprocess
import sys

packages = [
    'catboost',
    'optuna', 
    'lightgbm',
    'imbalanced-learn'
]

for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
        print(f"✅ {package} installed successfully")

print("\n📦 All required packages are now available!")
print("Available techniques:")
print("✓ SMOTE-ENN & SMOTE-Tomek for hybrid sampling")
print("✓ Optuna for Bayesian hyperparameter optimization") 
print("✓ LightGBM for fast gradient boosting")
print("✓ Advanced ensemble methods")
print("✓ Class weighting and sampling capabilities")

✅ catboost already installed
✅ optuna already installed
✅ lightgbm already installed
📦 Installing imbalanced-learn...
✅ imbalanced-learn installed successfully

📦 All required packages are now available!
Available techniques:
✓ SMOTE-ENN & SMOTE-Tomek for hybrid sampling
✓ Optuna for Bayesian hyperparameter optimization
✓ LightGBM for fast gradient boosting
✓ Advanced ensemble methods
✓ Class weighting and sampling capabilities


## 🎯 Class Imbalance Handling Techniques

We'll apply multiple class imbalance techniques to improve model performance:

### 1. **Hybrid Sampling Methods**
- **SMOTE-ENN**: Combines SMOTE oversampling with Edited Nearest Neighbours undersampling
- **SMOTE-Tomek**: Combines SMOTE oversampling with Tomek Links undersampling

### 2. **Algorithmic Approaches**
- **Class Weights**: Automatically balance classes in model training
- **Focal Loss**: Focus learning on hard-to-classify minority examples

### 3. **Advanced Models**
We'll test the top 5 performing algorithms:
- **Gradient Boosting Classifier** (ensemble method with boosting)
- **CatBoost Classifier** (gradient boosting with categorical features)
- **AdaBoost Classifier** (adaptive boosting algorithm)
- **LightGBM Classifier** (fast gradient boosting framework)
- **Logistic Regression** (linear model with regularization)

In [9]:
# Class Imbalance Handling Setup
print("🔧 Setting up class imbalance handling techniques...")

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
scale_pos_weight = class_weights[0] / class_weights[1]

print(f"Class weights: {class_weight_dict}")
print(f"Scale pos weight (XGBoost): {scale_pos_weight:.3f}")

# Initialize sampling techniques
sampling_techniques = {
    'original': None,
    'smote_enn': SMOTEENN(random_state=42, n_jobs=-1),
    'smote_tomek': SMOTETomek(random_state=42, n_jobs=-1),
    'smote_only': SMOTE(random_state=42, n_jobs=-1)
}

# Function to apply sampling
def apply_sampling(technique_name, X_train, y_train):
    if technique_name == 'original':
        return X_train, y_train
    else:
        technique = sampling_techniques[technique_name]
        X_resampled, y_resampled = technique.fit_resample(X_train, y_train)
        print(f"  {technique_name}: {Counter(y_train)} → {Counter(y_resampled)}")
        return X_resampled, y_resampled

# Extended hyperparameter search spaces for better performance
hyperparameter_spaces = {
    'gradientboosting': {
        'n_estimators': [200, 300, 500, 700, 1000, 1500],
        'max_depth': [3, 4, 5, 6, 7, 8, 10],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.3],
        'subsample': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],
        'max_features': ['sqrt', 'log2', 0.7, 0.8, 0.9, 1.0],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    },
    'catboost': {
        'iterations': [200, 300, 500, 700, 1000, 1500],
        'depth': [4, 5, 6, 7, 8, 9, 10, 12],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.3],
        'l2_leaf_reg': [1, 3, 5, 7, 9, 12, 15, 20],
        'border_count': [32, 64, 128, 200, 254],
        'bagging_temperature': [0, 0.5, 1.0, 2.0, 3.0],
        'random_strength': [0, 1, 2, 3, 5]
    },
    'adaboost': {
        'n_estimators': [100, 200, 300, 500, 700, 1000, 1500],
        'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1.0, 1.5, 2.0],
        'algorithm': ['SAMME', 'SAMME.R']
    },
    'lightgbm': {
        'n_estimators': [200, 300, 500, 700, 1000, 1500],
        'max_depth': [3, 4, 5, 6, 7, 8, 10, 12],
        'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15, 0.2, 0.3],
        'subsample': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.85, 0.9, 0.95, 1.0],
        'reg_alpha': [0, 0.1, 0.3, 0.5, 1.0, 2.0],
        'reg_lambda': [0.1, 0.3, 0.5, 1.0, 2.0, 3.0],
        'min_child_samples': [5, 10, 20, 30, 40, 50],
        'num_leaves': [31, 50, 70, 90, 110, 130]
    },
    'logisticregression': {
        'C': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [1000, 2000, 3000, 5000],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  # For elasticnet
    }
}

print("✅ Class imbalance techniques and extended hyperparameters ready!")

🔧 Setting up class imbalance handling techniques...
Class weights: {0: 0.6809927360774818, 1: 1.8812709030100334}
Scale pos weight (XGBoost): 0.362
✅ Class imbalance techniques and extended hyperparameters ready!


## 🔍 Advanced Model Optimization with Class Imbalance Handling

We'll use Optuna Bayesian optimization for efficient hyperparameter search combined with SMOTE-Tomek class imbalance technique.
Each model will be tested with:
1. **SMOTE-Tomek** (hybrid sampling combining oversampling and undersampling)

**Strategy**: 
- Optuna Bayesian optimization with **ultra-fast trial budget** (20 trials per model)
- **Pruning**: Early stopping of underperforming trials (MedianPruner)
- **Parallel processing**: Multi-core optimization (n_jobs=-1)
- **Warm start**: Faster gradient boosting training
- 5-fold Stratified Cross-Validation
- Primary metrics: **AUC-ROC**, **Accuracy**, **Precision**, **Recall**, **F1-Score**
- **SPEED OPTIMIZED** for fastest execution using single best sampling technique

In [10]:
# Optimized Optuna-based Hyperparameter Tuning Function with Speed Improvements
def optimize_model_with_optuna(model_name, model_class, param_space, X_train, y_train, 
                               sampling_technique='original', n_trials=200, class_weight_dict=None, n_jobs=-1):
    """
    Optimize model hyperparameters using Optuna with class imbalance handling and speed optimizations
    """
    print(f"\n🔍 Optimizing {model_name} with {sampling_technique} sampling...")
    
    # Dynamic trial budget based on sampling technique complexity (SPEED OPTIMIZED)
    trial_budget = {
        'smote_tomek': 20,      # Focused on single best hybrid sampling technique
    }
    actual_trials = trial_budget.get(sampling_technique, 20)
    print(f"   Using {actual_trials} trials (optimized budget)")
    
    # Apply sampling technique
    X_resampled, y_resampled = apply_sampling(sampling_technique, X_train, y_train)
    
    # Create stratified cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    def objective(trial):
        # Sample hyperparameters based on model type
        params = {}
        
        if model_name == 'LogisticRegression':
            params['C'] = trial.suggest_float('C', 0.01, 100.0, log=True)
            penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
            params['penalty'] = penalty
            
            if penalty == 'elasticnet':
                params['solver'] = 'saga'
                params['l1_ratio'] = trial.suggest_float('l1_ratio', 0.1, 0.9)
            elif penalty == 'l1':
                params['solver'] = trial.suggest_categorical('solver', ['liblinear', 'saga'])
            else:  # l2
                params['solver'] = trial.suggest_categorical('solver', ['liblinear', 'saga'])
            
            params['max_iter'] = trial.suggest_categorical('max_iter', [1000, 2000, 3000, 5000])
            
        else:
            # For tree-based models
            for param, values in param_space.items():
                if isinstance(values, list):
                    if all(isinstance(v, int) for v in values):
                        params[param] = trial.suggest_int(param, min(values), max(values))
                    elif all(isinstance(v, float) for v in values):
                        params[param] = trial.suggest_float(param, min(values), max(values))
                    else:
                        params[param] = trial.suggest_categorical(param, values)
        
        # Add class balancing parameters for original sampling
        if sampling_technique == 'original':
            if model_name == 'LogisticRegression':
                params['class_weight'] = class_weight_dict
            elif model_name == 'LightGBM':
                params['class_weight'] = class_weight_dict
            elif model_name == 'CatBoost':
                params['class_weights'] = [class_weight_dict[0], class_weight_dict[1]]
            # GradientBoosting and AdaBoost rely on sampling techniques
        
        # Add warm_start for gradient boosting models to speed up training
        if model_name == 'GradientBoosting':
            params['warm_start'] = True
        
        # Create model with sampled parameters
        try:
            if model_name == 'CatBoost':
                model = model_class(**params, random_state=42, verbose=False)
            elif model_name == 'LightGBM':
                model = model_class(**params, random_state=42, n_jobs=n_jobs, verbose=-1)
            elif model_name == 'LogisticRegression':
                model = model_class(**params, random_state=42, n_jobs=n_jobs)
            else:
                model = model_class(**params, random_state=42)
        except Exception as e:
            return 0.0
        
        # Perform cross-validation with parallel processing
        try:
            scores = cross_val_score(model, X_resampled, y_resampled, cv=cv, 
                                   scoring='roc_auc', n_jobs=n_jobs)
            return scores.mean()
        except Exception as e:
            return 0.0
    
    # Create study with pruning for early stopping of underperforming trials
    study = optuna.create_study(
        direction='maximize',
        study_name=f"{model_name}_{sampling_technique}",
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5)
    )
    
    # Run optimization with parallel trials
    study.optimize(objective, n_trials=actual_trials, show_progress_bar=True, n_jobs=min(4, n_jobs) if n_jobs > 0 else 4)
    
    # Get best parameters and create best model
    best_params = study.best_params.copy()
    
    # Add class balancing to best params for original sampling
    if sampling_technique == 'original':
        if model_name == 'LogisticRegression':
            best_params['class_weight'] = class_weight_dict
        elif model_name == 'LightGBM':
            best_params['class_weight'] = class_weight_dict
        elif model_name == 'CatBoost':
            best_params['class_weights'] = [class_weight_dict[0], class_weight_dict[1]]
    
    # Remove warm_start from final model params as it's only for training optimization
    if 'warm_start' in best_params:
        del best_params['warm_start']
    
    # Create and evaluate best model
    if model_name == 'CatBoost':
        best_model = model_class(**best_params, random_state=42, verbose=False)
    elif model_name == 'LightGBM':
        best_model = model_class(**best_params, random_state=42, n_jobs=n_jobs, verbose=-1)
    elif model_name == 'LogisticRegression':
        best_model = model_class(**best_params, random_state=42, n_jobs=n_jobs)
    else:
        best_model = model_class(**best_params, random_state=42)
    
    # Get comprehensive cross-validation scores with parallel processing
    cv_results = {}
    for metric in ['roc_auc', 'accuracy', 'precision', 'recall', 'f1']:
        scores = cross_val_score(best_model, X_resampled, y_resampled, cv=cv, 
                               scoring=metric, n_jobs=n_jobs)
        cv_results[metric] = {'mean': scores.mean(), 'std': scores.std()}
    
    return {
        'model': best_model,
        'best_params': best_params,
        'best_score': study.best_value,
        'cv_results': cv_results,
        'study': study,
        'sampling_data': (X_resampled, y_resampled),
        'trials_used': actual_trials,
        'pruned_trials': len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
    }

print("✅ Optimized Optuna function ready with MAXIMUM SPEED!")
print("🚀 Features: Ultra-fast trial budgets, pruning, parallel processing, warm start")
print("📝 Trial budget: smote_tomek=20 (focused approach)")

✅ Optimized Optuna function ready with MAXIMUM SPEED!
🚀 Features: Ultra-fast trial budgets, pruning, parallel processing, warm start
📝 Trial budget: smote_tomek=20 (focused approach)


## 🎯 Model 1: Gradient Boosting Classifier Optimization

Optimizing the best performing model from baseline analysis with comprehensive class imbalance techniques.

In [11]:
# Model 1: Gradient Boosting Classifier
import time
from datetime import datetime

print("🚀 OPTIMIZING GRADIENT BOOSTING CLASSIFIER")
print("="*60)

# Define model and sampling techniques
model_name = 'GradientBoosting'
model_class = GradientBoostingClassifier
sampling_methods = ['smote_tomek']  # Using only SMOTE-Tomek for speed

# Store results for this model
gb_results = {}
gb_times = {}

for sampling_method in sampling_methods:
    start_time = time.time()
    
    try:
        # Get hyperparameter space
        param_space = hyperparameter_spaces['gradientboosting']
        
        # Run optimization with speed improvements
        result = optimize_model_with_optuna(
            model_name=model_name,
            model_class=model_class,
            param_space=param_space,
            X_train=X_train,
            y_train=y_train,
            sampling_technique=sampling_method,
            class_weight_dict=class_weight_dict,
            n_jobs=-1  # Use all available CPU cores
        )
        
        gb_results[sampling_method] = result
        gb_times[sampling_method] = time.time() - start_time
        
        # Print results with optimization stats
        cv_results = result['cv_results']
        print(f"\n✅ {sampling_method.upper()}:")
        print(f"   AUC-ROC: {cv_results['roc_auc']['mean']:.4f} (±{cv_results['roc_auc']['std']:.4f})")
        print(f"   Accuracy: {cv_results['accuracy']['mean']:.4f} (±{cv_results['accuracy']['std']:.4f})")
        print(f"   Precision: {cv_results['precision']['mean']:.4f} (±{cv_results['precision']['std']:.4f})")
        print(f"   Recall: {cv_results['recall']['mean']:.4f} (±{cv_results['recall']['std']:.4f})")
        print(f"   F1-Score: {cv_results['f1']['mean']:.4f} (±{cv_results['f1']['std']:.4f})")
        print(f"   Trials: {result['trials_used']} | Pruned: {result['pruned_trials']} | Time: {gb_times[sampling_method]:.1f}s")
        
    except Exception as e:
        print(f"❌ {sampling_method}: Optimization failed - {str(e)}")
        gb_results[sampling_method] = None
        gb_times[sampling_method] = time.time() - start_time

# Find best configuration for Gradient Boosting
best_gb_method = max([k for k, v in gb_results.items() if v is not None], 
                     key=lambda x: gb_results[x]['cv_results']['accuracy']['mean'])
best_gb_result = gb_results[best_gb_method]

print(f"\n🏆 BEST GRADIENT BOOSTING CONFIGURATION:")
print(f"Sampling Method: {best_gb_method}")
print(f"Best Accuracy: {best_gb_result['cv_results']['accuracy']['mean']:.4f}")
print(f"Best AUC-ROC: {best_gb_result['cv_results']['roc_auc']['mean']:.4f}")
print(f"Total time: {sum(gb_times.values())/60:.1f} minutes")

[I 2025-07-01 15:53:21,431] A new study created in memory with name: GradientBoosting_smote_tomek


🚀 OPTIMIZING GRADIENT BOOSTING CLASSIFIER

🔍 Optimizing GradientBoosting with smote_tomek sampling...
   Using 20 trials (optimized budget)
  smote_tomek: Counter({0: 3304, 1: 1196}) → Counter({0: 2929, 1: 2929})


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-07-01 15:53:31,034] Trial 1 finished with value: 0.9377457303536232 and parameters: {'n_estimators': 221, 'max_depth': 3, 'learning_rate': 0.055161648382533686, 'subsample': 0.9294052864169349, 'max_features': 0.8, 'min_samples_split': 7, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.9377457303536232.
[I 2025-07-01 15:53:42,145] Trial 4 finished with value: 0.938593382457551 and parameters: {'n_estimators': 360, 'max_depth': 4, 'learning_rate': 0.2997322374557249, 'subsample': 0.7947967106684836, 'max_features': 0.8, 'min_samples_split': 15, 'min_samples_leaf': 3}. Best is trial 4 with value: 0.938593382457551.
[I 2025-07-01 15:53:53,315] Trial 2 finished with value: 0.9359169747757268 and parameters: {'n_estimators': 1326, 'max_depth': 10, 'learning_rate': 0.11311666281124891, 'subsample': 0.8398590991628633, 'max_features': 'sqrt', 'min_samples_split': 13, 'min_samples_leaf': 5}. Best is trial 4 with value: 0.938593382457551.
[I 2025-07-01 15:54:01,167] Trial 6 finish

## 🎯 Model 2: CatBoost Classifier Optimization

Optimizing the second-best performing model with advanced categorical feature handling and gradient boosting techniques.

In [12]:
# Model 2: CatBoost Classifier
print("🚀 OPTIMIZING CATBOOST CLASSIFIER")
print("="*60)

# Define model and sampling techniques
model_name = 'CatBoost'
model_class = CatBoostClassifier
sampling_methods = ['smote_tomek']  # Using only SMOTE-Tomek for speed

# Store results for this model
catboost_results = {}
catboost_times = {}

for sampling_method in sampling_methods:
    start_time = time.time()
    
    try:
        # Get hyperparameter space
        param_space = hyperparameter_spaces['catboost']
        
        # Run optimization with speed improvements
        result = optimize_model_with_optuna(
            model_name=model_name,
            model_class=model_class,
            param_space=param_space,
            X_train=X_train,
            y_train=y_train,
            sampling_technique=sampling_method,
            class_weight_dict=class_weight_dict,
            n_jobs=-1  # Use all available CPU cores
        )
        
        catboost_results[sampling_method] = result
        catboost_times[sampling_method] = time.time() - start_time
        
        # Print results with optimization stats
        cv_results = result['cv_results']
        print(f"\n✅ {sampling_method.upper()}:")
        print(f"   AUC-ROC: {cv_results['roc_auc']['mean']:.4f} (±{cv_results['roc_auc']['std']:.4f})")
        print(f"   Accuracy: {cv_results['accuracy']['mean']:.4f} (±{cv_results['accuracy']['std']:.4f})")
        print(f"   Precision: {cv_results['precision']['mean']:.4f} (±{cv_results['precision']['std']:.4f})")
        print(f"   Recall: {cv_results['recall']['mean']:.4f} (±{cv_results['recall']['std']:.4f})")
        print(f"   F1-Score: {cv_results['f1']['mean']:.4f} (±{cv_results['f1']['std']:.4f})")
        print(f"   Trials: {result['trials_used']} | Pruned: {result['pruned_trials']} | Time: {catboost_times[sampling_method]:.1f}s")
        
    except Exception as e:
        print(f"❌ {sampling_method}: Optimization failed - {str(e)}")
        catboost_results[sampling_method] = None
        catboost_times[sampling_method] = time.time() - start_time

# Find best configuration for CatBoost
best_catboost_method = max([k for k, v in catboost_results.items() if v is not None], 
                          key=lambda x: catboost_results[x]['cv_results']['accuracy']['mean'])
best_catboost_result = catboost_results[best_catboost_method]

print(f"\n🏆 BEST CATBOOST CONFIGURATION:")
print(f"Sampling Method: {best_catboost_method}")
print(f"Best Accuracy: {best_catboost_result['cv_results']['accuracy']['mean']:.4f}")
print(f"Best AUC-ROC: {best_catboost_result['cv_results']['roc_auc']['mean']:.4f}")
print(f"Total time: {sum(catboost_times.values())/60:.1f} minutes")

[I 2025-07-01 15:55:52,775] A new study created in memory with name: CatBoost_smote_tomek


🚀 OPTIMIZING CATBOOST CLASSIFIER

🔍 Optimizing CatBoost with smote_tomek sampling...
   Using 20 trials (optimized budget)
  smote_tomek: Counter({0: 3304, 1: 1196}) → Counter({0: 2929, 1: 2929})


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-07-01 15:56:36,521] Trial 3 finished with value: 0.9398791124308602 and parameters: {'iterations': 942, 'depth': 6, 'learning_rate': 0.20917106003129393, 'l2_leaf_reg': 8, 'border_count': 141, 'bagging_temperature': 0, 'random_strength': 0}. Best is trial 3 with value: 0.9398791124308602.
[I 2025-07-01 15:56:48,571] Trial 2 finished with value: 0.9409294235864143 and parameters: {'iterations': 658, 'depth': 8, 'learning_rate': 0.26204081307778715, 'l2_leaf_reg': 16, 'border_count': 124, 'bagging_temperature': 2.0, 'random_strength': 5}. Best is trial 2 with value: 0.9409294235864143.
[I 2025-07-01 15:57:03,795] Trial 1 finished with value: 0.9395947645304072 and parameters: {'iterations': 612, 'depth': 10, 'learning_rate': 0.22810010209863743, 'l2_leaf_reg': 5, 'border_count': 61, 'bagging_temperature': 3.0, 'random_strength': 0}. Best is trial 2 with value: 0.9409294235864143.
[I 2025-07-01 15:57:32,218] Trial 4 finished with value: 0.9377502493196911 and parameters: {'iterati

## 🎯 Model 3: AdaBoost Classifier Optimization

Optimizing the adaptive boosting algorithm with advanced class imbalance handling for improved minority class prediction.

In [13]:
# Model 3: AdaBoost Classifier
print("🚀 OPTIMIZING ADABOOST CLASSIFIER")
print("="*60)

# Define model and sampling techniques
model_name = 'AdaBoost'
model_class = AdaBoostClassifier
sampling_methods = ['smote_tomek']  # Using only SMOTE-Tomek for speed

# Store results for this model
adaboost_results = {}
adaboost_times = {}

for sampling_method in sampling_methods:
    start_time = time.time()
    
    try:
        # Get hyperparameter space
        param_space = hyperparameter_spaces['adaboost']
        
        # Run optimization with speed improvements
        result = optimize_model_with_optuna(
            model_name=model_name,
            model_class=model_class,
            param_space=param_space,
            X_train=X_train,
            y_train=y_train,
            sampling_technique=sampling_method,
            class_weight_dict=class_weight_dict,
            n_jobs=-1  # Use all available CPU cores
        )
        
        adaboost_results[sampling_method] = result
        adaboost_times[sampling_method] = time.time() - start_time
        
        # Print results with optimization stats
        cv_results = result['cv_results']
        print(f"\n✅ {sampling_method.upper()}:")
        print(f"   AUC-ROC: {cv_results['roc_auc']['mean']:.4f} (±{cv_results['roc_auc']['std']:.4f})")
        print(f"   Accuracy: {cv_results['accuracy']['mean']:.4f} (±{cv_results['accuracy']['std']:.4f})")
        print(f"   Precision: {cv_results['precision']['mean']:.4f} (±{cv_results['precision']['std']:.4f})")
        print(f"   Recall: {cv_results['recall']['mean']:.4f} (±{cv_results['recall']['std']:.4f})")
        print(f"   F1-Score: {cv_results['f1']['mean']:.4f} (±{cv_results['f1']['std']:.4f})")
        print(f"   Trials: {result['trials_used']} | Pruned: {result['pruned_trials']} | Time: {adaboost_times[sampling_method]:.1f}s")
        
    except Exception as e:
        print(f"❌ {sampling_method}: Optimization failed - {str(e)}")
        adaboost_results[sampling_method] = None
        adaboost_times[sampling_method] = time.time() - start_time

# Find best configuration for AdaBoost
best_adaboost_method = max([k for k, v in adaboost_results.items() if v is not None], 
                          key=lambda x: adaboost_results[x]['cv_results']['accuracy']['mean'])
best_adaboost_result = adaboost_results[best_adaboost_method]

print(f"\n🏆 BEST ADABOOST CONFIGURATION:")
print(f"Sampling Method: {best_adaboost_method}")
print(f"Best Accuracy: {best_adaboost_result['cv_results']['accuracy']['mean']:.4f}")
print(f"Best AUC-ROC: {best_adaboost_result['cv_results']['roc_auc']['mean']:.4f}")
print(f"Total time: {sum(adaboost_times.values())/60:.1f} minutes")

[I 2025-07-01 16:44:17,619] A new study created in memory with name: AdaBoost_smote_tomek


🚀 OPTIMIZING ADABOOST CLASSIFIER

🔍 Optimizing AdaBoost with smote_tomek sampling...
   Using 20 trials (optimized budget)
  smote_tomek: Counter({0: 3304, 1: 1196}) → Counter({0: 2929, 1: 2929})


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-07-01 16:44:25,263] Trial 2 finished with value: 0.9349729519122872 and parameters: {'n_estimators': 383, 'learning_rate': 1.6198964781600413, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.9349729519122872.
[I 2025-07-01 16:44:29,189] Trial 4 finished with value: 0.9292149244753236 and parameters: {'n_estimators': 283, 'learning_rate': 1.2103830686285866, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.9349729519122872.
[I 2025-07-01 16:44:30,252] Trial 3 finished with value: 0.9333498301977843 and parameters: {'n_estimators': 781, 'learning_rate': 1.2845599554892917, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.9349729519122872.
[I 2025-07-01 16:44:31,525] Trial 0 finished with value: 0.933031425282296 and parameters: {'n_estimators': 872, 'learning_rate': 1.283889166562746, 'algorithm': 'SAMME'}. Best is trial 2 with value: 0.9349729519122872.
[I 2025-07-01 16:44:39,299] Trial 1 finished with value: 0.932570891964653 and parameters: {'n_estimators': 1445, 

## 📊 Comprehensive Results Analysis & Model Comparison

Analyzing results from top 3 models with SMOTE-Tomek sampling (3 total configurations) to identify the best performer and create ensemble models.

In [14]:
# Consolidate all results from first 3 models (optimized for speed)
all_results = {
    'GradientBoosting': gb_results,
    'CatBoost': catboost_results,
    'AdaBoost': adaboost_results
}

optimization_times = {
    'GradientBoosting': gb_times,
    'CatBoost': catboost_times,
    'AdaBoost': adaboost_times
}

# Create comprehensive results DataFrame
results_data = []

for model_name, model_results in all_results.items():
    for sampling_method, result in model_results.items():
        if result is not None:
            cv_results = result['cv_results']
            results_data.append({
                'Model': model_name,
                'Sampling': sampling_method,
                'AUC-ROC': cv_results['roc_auc']['mean'],
                'AUC-ROC_std': cv_results['roc_auc']['std'],
                'Accuracy': cv_results['accuracy']['mean'],
                'Accuracy_std': cv_results['accuracy']['std'],
                'Precision': cv_results['precision']['mean'],
                'Precision_std': cv_results['precision']['std'],
                'Recall': cv_results['recall']['mean'],
                'Recall_std': cv_results['recall']['std'],
                'F1': cv_results['f1']['mean'],
                'F1_std': cv_results['f1']['std'],
                'Training_Time': optimization_times[model_name][sampling_method],
                'Best_Params': str(result['best_params'])
            })

results_df = pd.DataFrame(results_data)

# Sort by accuracy (primary metric)
results_df_sorted = results_df.sort_values('Accuracy', ascending=False)

print("🏆 TOP PERFORMING CONFIGURATIONS:")
print("="*100)
top_configs = results_df_sorted.head(3)  # Show all configurations from 3 models
for idx, row in top_configs.iterrows():
    print(f"{row['Model']:18} + {row['Sampling']:12} | "
          f"Acc: {row['Accuracy']:.4f} (±{row['Accuracy_std']:.4f}) | "
          f"AUC: {row['AUC-ROC']:.4f} (±{row['AUC-ROC_std']:.4f}) | "
          f"F1: {row['F1']:.4f} | "
          f"Time: {row['Training_Time']:.0f}s")

# Find best overall configuration
best_config = results_df_sorted.iloc[0]
print(f"\n🎯 BEST INDIVIDUAL MODEL CONFIGURATION:")
print(f"Model: {best_config['Model']}")
print(f"Sampling: {best_config['Sampling']}")
print(f"Accuracy: {best_config['Accuracy']:.4f} ± {best_config['Accuracy_std']:.4f}")
print(f"AUC-ROC: {best_config['AUC-ROC']:.4f} ± {best_config['AUC-ROC_std']:.4f}")
print(f"Precision: {best_config['Precision']:.4f} ± {best_config['Precision_std']:.4f}")
print(f"Recall: {best_config['Recall']:.4f} ± {best_config['Recall_std']:.4f}")
print(f"F1-Score: {best_config['F1']:.4f} ± {best_config['F1_std']:.4f}")
print(f"Training Time: {best_config['Training_Time']:.1f} seconds")

# Get best configuration for each model
best_models = {}
for model_name in all_results.keys():
    model_results = results_df[results_df['Model'] == model_name]
    if not model_results.empty:
        best_idx = model_results['Accuracy'].idxmax()
        best_models[model_name] = model_results.loc[best_idx]

print(f"\n📈 BEST CONFIGURATION FOR EACH MODEL:")
print("="*80)
for model_name, best_result in best_models.items():
    print(f"{model_name:18}: {best_result['Sampling']:12} | "
          f"Acc: {best_result['Accuracy']:.4f} | "
          f"AUC: {best_result['AUC-ROC']:.4f}")

# Save results summary
results_df.to_csv('../Data/output/advanced_optimization_results.csv', index=False)
print(f"\n💾 Results saved to advanced_optimization_results.csv")

🏆 TOP PERFORMING CONFIGURATIONS:
GradientBoosting   + smote_tomek  | Acc: 0.8709 (±0.0074) | AUC: 0.9444 (±0.0024) | F1: 0.8723 | Time: 151s
CatBoost           + smote_tomek  | Acc: 0.8670 (±0.0075) | AUC: 0.9424 (±0.0030) | F1: 0.8676 | Time: 2905s
AdaBoost           + smote_tomek  | Acc: 0.8532 (±0.0043) | AUC: 0.9368 (±0.0027) | F1: 0.8567 | Time: 116s

🎯 BEST INDIVIDUAL MODEL CONFIGURATION:
Model: GradientBoosting
Sampling: smote_tomek
Accuracy: 0.8709 ± 0.0074
AUC-ROC: 0.9444 ± 0.0024
Precision: 0.8632 ± 0.0051
Recall: 0.8815 ± 0.0115
F1-Score: 0.8723 ± 0.0078
Training Time: 151.3 seconds

📈 BEST CONFIGURATION FOR EACH MODEL:
GradientBoosting  : smote_tomek  | Acc: 0.8709 | AUC: 0.9444
CatBoost          : smote_tomek  | Acc: 0.8670 | AUC: 0.9424
AdaBoost          : smote_tomek  | Acc: 0.8532 | AUC: 0.9368

💾 Results saved to advanced_optimization_results.csv


## 🤝 Ensemble Methods (Speed Optimized)

Combining the top 2 diverse individual models using soft voting for fast execution while maintaining high performance.

### Ensemble Technique Applied:
1. **Soft Voting**: Average of predicted probabilities from top 2 diverse models

### Speed Optimizations:
- **Top 2 Models Only**: Reduced from 3 to 2 models for faster training
- **cross_validate()**: Single call for multiple metrics instead of multiple cross_val_score() calls
- **CV=5 with Stratification**: Efficient 5-fold stratified cross-validation
- **Validation Set Confirmation**: Final performance check on held-out data

### Expected Benefits:
- Reduced overfitting through model diversity
- Better generalization performance  
- Improved prediction stability
- **Ultra-fast execution** (1-2 minutes)

In [19]:
# Ensemble Methods Implementation
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

print("🤝 ENSEMBLE METHODS EVALUATION")
print("="*60)

# Get best accuracy from our results for comparison
best_accuracy = best_config['Accuracy']

print(f"🎯 Current best individual model accuracy: {best_accuracy:.4f}")

# Get top 2 best configurations for faster ensemble (diverse models)
top_2_configs = results_df_sorted.head(2)
print("\n🏆 Top 2 configurations for ensemble (optimized for speed):")
for idx, config in top_2_configs.iterrows():
    print(f"  {config['Model']} + {config['Sampling']}: {config['Accuracy']:.4f}")

# Extract best models for ensemble
ensemble_models = []
ensemble_data = None

for idx, config in top_2_configs.iterrows():
    model_name = config['Model']
    sampling_method = config['Sampling']
    result = all_results[model_name][sampling_method]
    
    if result is not None:
        # Get the optimized model
        model = result['model']
        X_ensemble, y_ensemble = result['sampling_data']
        
        ensemble_models.append((f"{model_name}_{sampling_method}", model))
        if ensemble_data is None:
            ensemble_data = (X_ensemble, y_ensemble)

print(f"\n🔧 Creating ensemble with {len(ensemble_models)} models...")

# Create ensemble methods (optimized for speed)
from sklearn.model_selection import cross_validate

ensemble_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Soft Voting Classifier (Fast execution - only method used)
soft_voting_clf = VotingClassifier(
    estimators=ensemble_models,
    voting='soft',
    n_jobs=-1
)

print(f"\n🔍 Evaluating Soft Voting Ensemble (Fast Mode)...")

# Use cross_validate for efficient multiple metrics evaluation
scoring = ['roc_auc', 'accuracy', 'precision', 'recall', 'f1']
cv_results_raw = cross_validate(
    soft_voting_clf, 
    ensemble_data[0], 
    ensemble_data[1], 
    cv=cv, 
    scoring=scoring, 
    n_jobs=-1,
    return_train_score=False
)

# Process results
cv_scores = {}
for metric in scoring:
    scores = cv_results_raw[f'test_{metric}']
    cv_scores[metric] = {'mean': scores.mean(), 'std': scores.std()}

ensemble_results['Soft_Voting'] = cv_scores

print(f"  AUC-ROC: {cv_scores['roc_auc']['mean']:.4f} ± {cv_scores['roc_auc']['std']:.4f}")
print(f"  Accuracy: {cv_scores['accuracy']['mean']:.4f} ± {cv_scores['accuracy']['std']:.4f}")
print(f"  Precision: {cv_scores['precision']['mean']:.4f} ± {cv_scores['precision']['std']:.4f}")
print(f"  Recall: {cv_scores['recall']['mean']:.4f} ± {cv_scores['recall']['std']:.4f}")
print(f"  F1-Score: {cv_scores['f1']['mean']:.4f} ± {cv_scores['f1']['std']:.4f}")

# Validate on held-out validation set for final confirmation
print(f"\n🧪 Validating ensemble on held-out validation set...")
soft_voting_clf.fit(ensemble_data[0], ensemble_data[1])
val_pred = soft_voting_clf.predict(X_val)
val_pred_proba = soft_voting_clf.predict_proba(X_val)[:, 1]

val_accuracy = accuracy_score(y_val, val_pred)
val_auc = roc_auc_score(y_val, val_pred_proba)

print(f"  Validation Accuracy: {val_accuracy:.4f}")
print(f"  Validation AUC-ROC: {val_auc:.4f}")

print(f"\n✅ Ensemble evaluation completed efficiently! (Top 2 models, cross_validate)")
print(f"📊 CV vs Validation: Acc {cv_scores['accuracy']['mean']:.4f} vs {val_accuracy:.4f}")

🤝 ENSEMBLE METHODS EVALUATION
🎯 Current best individual model accuracy: 0.8709

🏆 Top 2 configurations for ensemble (optimized for speed):
  GradientBoosting + smote_tomek: 0.8709
  CatBoost + smote_tomek: 0.8670

🔧 Creating ensemble with 2 models...

🔍 Evaluating Soft Voting Ensemble (Fast Mode)...
  AUC-ROC: 0.9449 ± 0.0022
  Accuracy: 0.8699 ± 0.0075
  Precision: 0.8632 ± 0.0067
  Recall: 0.8791 ± 0.0116
  F1-Score: 0.8711 ± 0.0078

🧪 Validating ensemble on held-out validation set...
  Validation Accuracy: 0.7707
  Validation AUC-ROC: 0.8096

✅ Ensemble evaluation completed efficiently! (Top 2 models, cross_validate)
📊 CV vs Validation: Acc 0.8699 vs 0.7707


## 📊 Comprehensive Results Comparison

Comparing all individual models and ensemble methods to select the best performing approach for final model selection.

In [20]:
# Comprehensive Results Comparison
print("📊 COMPREHENSIVE RESULTS COMPARISON")
print("="*80)

# Combine individual and ensemble results
all_methods = []

# Add individual model results
for idx, config in results_df_sorted.iterrows():
    all_methods.append({
        'Method': f"{config['Model']} ({config['Sampling']})",
        'Type': 'Individual',
        'AUC-ROC': config['AUC-ROC'],
        'Accuracy': config['Accuracy'],
        'Precision': config['Precision'],
        'Recall': config['Recall'],
        'F1': config['F1']
    })

# Add ensemble results
for name, results in ensemble_results.items():
    all_methods.append({
        'Method': f"{name} Ensemble",
        'Type': 'Ensemble',
        'AUC-ROC': results['roc_auc']['mean'],
        'Accuracy': results['accuracy']['mean'],
        'Precision': results['precision']['mean'],
        'Recall': results['recall']['mean'],
        'F1': results['f1']['mean']
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(all_methods)
comparison_df = comparison_df.sort_values('Accuracy', ascending=False)

print("\n🏆 FINAL RANKINGS BY ACCURACY:")
print("-" * 80)
for idx, row in comparison_df.head(10).iterrows():
    print(f"{idx+1:2d}. {row['Method']:35} | {row['Type']:10} | "
          f"Acc: {row['Accuracy']:.4f} | AUC: {row['AUC-ROC']:.4f} | F1: {row['F1']:.4f}")

# Find best overall method
best_method = comparison_df.iloc[0]
print(f"\n🥇 BEST OVERALL METHOD:")
print(f"Method: {best_method['Method']}")
print(f"Type: {best_method['Type']}")
print(f"Accuracy: {best_method['Accuracy']:.4f}")
print(f"AUC-ROC: {best_method['AUC-ROC']:.4f}")
print(f"Precision: {best_method['Precision']:.4f}")
print(f"Recall: {best_method['Recall']:.4f}")
print(f"F1-Score: {best_method['F1']:.4f}")

# Determine final model for saving
if best_method['Type'] == 'Ensemble':
    # Use soft voting ensemble (only ensemble method)
    final_model = soft_voting_clf
    final_model_name = best_method['Method']
    final_training_data = ensemble_data
else:
    # Use best individual model
    model_name = best_config['Model']
    sampling_method = best_config['Sampling']
    result = all_results[model_name][sampling_method]
    
    final_model = result['model']
    final_model_name = f"{model_name} ({sampling_method})"
    final_training_data = result['sampling_data']

print(f"\n🎯 FINAL MODEL SELECTED: {final_model_name}")
print(f"Final Model Type: {type(final_model).__name__}")

📊 COMPREHENSIVE RESULTS COMPARISON

🏆 FINAL RANKINGS BY ACCURACY:
--------------------------------------------------------------------------------
 1. GradientBoosting (smote_tomek)      | Individual | Acc: 0.8709 | AUC: 0.9444 | F1: 0.8723
 4. Soft_Voting Ensemble                | Ensemble   | Acc: 0.8699 | AUC: 0.9449 | F1: 0.8711
 2. CatBoost (smote_tomek)              | Individual | Acc: 0.8670 | AUC: 0.9424 | F1: 0.8676
 3. AdaBoost (smote_tomek)              | Individual | Acc: 0.8532 | AUC: 0.9368 | F1: 0.8567

🥇 BEST OVERALL METHOD:
Method: GradientBoosting (smote_tomek)
Type: Individual
Accuracy: 0.8709
AUC-ROC: 0.9444
Precision: 0.8632
Recall: 0.8815
F1-Score: 0.8723

🎯 FINAL MODEL SELECTED: GradientBoosting (smote_tomek)
Final Model Type: GradientBoostingClassifier


## 💾 Final Model 



In [24]:
import os
import json
import pickle
from datetime import datetime
from pathlib import Path

print("💾 SAVING BEST MODEL WITH COMPREHENSIVE METADATA")
print("="*60)

# Define save directory and file paths
model_save_dir = Path("../Data/output")
model_save_dir.mkdir(parents=True, exist_ok=True)

model_pkl_path = model_save_dir / "best_model_optimized.pkl"
metadata_json_path = model_save_dir / "best_model_metadata.json"

# Get current working directory and data paths
current_dir = Path.cwd()
data_dir = Path("../Data")

# Comprehensive training data directory information
training_data_info = {
    "primary_training_file": {
        "path": "../Data/output/feature_engineered_train.csv",
        "absolute_path": str((current_dir / "../Data/output/feature_engineered_train.csv").resolve()),
        "exists": (current_dir / "../Data/output/feature_engineered_train.csv").exists(),
        "description": "Main feature-engineered training dataset used for optimization"
    },
    "data_directories": {
        "input_dir": {
            "path": "../Data/input/",
            "absolute_path": str((current_dir / "../Data/input").resolve()),
            "exists": (current_dir / "../Data/input").exists(),
            "description": "Original raw data directory"
        },
        "interim_dir": {
            "path": "../Data/interim/",
            "absolute_path": str((current_dir / "../Data/interim").resolve()),
            "exists": (current_dir / "../Data/interim").exists(),
            "description": "Intermediate processed data and model files"
        },
        "output_dir": {
            "path": "../Data/output/",
            "absolute_path": str((current_dir / "../Data/output").resolve()),
            "exists": (current_dir / "../Data/output").exists(),
            "description": "Final processed data and output files"
        }
    },
    "related_files": {
        "original_data": "../Data/input/Telco_Customer_kaggle.csv",
        "test_data": "../Data/output/test.csv",
        "feature_selection": "../Data/output/feature_selection_summary.csv"
    }
}

# Print essential training data information
print(f"\n📂 TRAINING DATA USED:")
print(f"Primary File: {training_data_info['primary_training_file']['path']}")
print(f"Data saved to metadata for complete directory tracking")

# Prepare comprehensive model metadata
model_metadata = {
    "model_info": {
        "model_name": final_model_name,
        "model_type": type(final_model).__name__,
        "optimization_timestamp": datetime.now().isoformat(),
        "python_version": sys.version,
        "optimization_method": "Optuna Bayesian + SMOTE-Tomek"
    },
    "performance_metrics": {
        "accuracy": float(best_method['Accuracy']),
        "auc_roc": float(best_method['AUC-ROC']),
        "precision": float(best_method['Precision']),
        "recall": float(best_method['Recall']),
        "f1_score": float(best_method['F1'])
    },
    "training_data": training_data_info,
    "model_configuration": {
        "cross_validation_folds": 5,
        "sampling_technique": "SMOTE-Tomek" if 'smote_tomek' in final_model_name.lower() else "Original",
        "hyperparameter_trials": 20,
        "ensemble_method": "Soft Voting" if "Ensemble" in final_model_name else "Single Model"
    },
    "data_shape": {
        "original_features": X.shape[1],
        "original_samples": X.shape[0],
        "training_samples": final_training_data[0].shape[0],
        "validation_samples": X_val.shape[0]
    },
    "class_distribution": {
        "original": {str(k): int(v) for k, v in Counter(y).items()},
        "training_resampled": {str(k): int(v) for k, v in Counter(final_training_data[1]).items()}
    },
    "file_paths": {
        "model_pkl": str(model_pkl_path.resolve()),
        "metadata_json": str(metadata_json_path.resolve()),
        "notebook_path": str(Path.cwd().resolve())
    }
}

# Save the model
print(f"\n💾 Saving optimized model...")
with open(model_pkl_path, 'wb') as f:
    pickle.dump(final_model, f)
print(f"✅ Model saved: {model_pkl_path.resolve()}")

# Save metadata
print(f"\n📋 Saving model metadata...")
with open(metadata_json_path, 'w') as f:
    json.dump(model_metadata, f, indent=2, default=str)
print(f"✅ Metadata saved: {metadata_json_path.resolve()}")

# Verify saved files
print(f"\n🔍 VERIFICATION:")
print(f"Model file size: {model_pkl_path.stat().st_size / 1024:.1f} KB")
print(f"Metadata file size: {metadata_json_path.stat().st_size / 1024:.1f} KB")

# Test loading the saved model
print(f"\n🧪 Testing model loading...")
with open(model_pkl_path, 'rb') as f:
    loaded_model = pickle.load(f)
print(f"✅ Model loaded successfully: {type(loaded_model).__name__}")

print(f"\n🎯 MODEL PERSISTENCE COMPLETE!")
print(f"📁 Files ready for use:")
print(f"   • Model: {model_pkl_path.name}")
print(f"   • Metadata: {metadata_json_path.name}")
print(f"   • Location: {model_save_dir.resolve()}")
print(f"📊 Best Performance: {best_method['Accuracy']:.4f} accuracy, {best_method['AUC-ROC']:.4f} AUC-ROC")

💾 SAVING BEST MODEL WITH COMPREHENSIVE METADATA

📂 TRAINING DATA USED:
Primary File: ../Data/output/feature_engineered_train.csv
Data saved to metadata for complete directory tracking

💾 Saving optimized model...
✅ Model saved: D:\telco-customer-churn-prediction-main\telco-customer-churn-prediction-main\Data\output\best_model_optimized.pkl

📋 Saving model metadata...
✅ Metadata saved: D:\telco-customer-churn-prediction-main\telco-customer-churn-prediction-main\Data\output\best_model_metadata.json

🔍 VERIFICATION:
Model file size: 5003.2 KB
Metadata file size: 2.9 KB

🧪 Testing model loading...
✅ Model loaded successfully: GradientBoostingClassifier

🎯 MODEL PERSISTENCE COMPLETE!
📁 Files ready for use:
   • Model: best_model_optimized.pkl
   • Metadata: best_model_metadata.json
   • Location: D:\telco-customer-churn-prediction-main\telco-customer-churn-prediction-main\Data\output
📊 Best Performance: 0.8709 accuracy, 0.9444 AUC-ROC


## 🚀 Model Usage Instructions

The optimized model has been saved and is ready for use with test data. Here's how to use the saved model:

### Loading the Model:
```python
import pickle
import json

# Load the model
with open('../Data/output/best_model_optimized.pkl', 'rb') as f:
    model = pickle.load(f)

# Load metadata for reference
with open('../Data/output/best_model_metadata.json', 'r') as f:
    metadata = json.load(f)
```

### Key Files Saved:
- **`best_model_optimized.pkl`**: The trained model ready for predictions
- **`best_model_metadata.json`**: Comprehensive metadata including:
  - Model performance metrics
  - Training data directories and paths
  - Hyperparameter optimization details
  - Data preprocessing information
  - Class distribution details

### Next Steps:
1. Load test data from the same feature engineering pipeline
2. Apply the same preprocessing (SMOTE-Tomek if ensemble)
3. Use `model.predict()` or `model.predict_proba()` for predictions
4. Evaluate final performance on test set